<h1>FCTM</h1>

<p><img alt="" src="https://www.nyu.edu/projects/yzhang/AlphaSpace/images/slide03.png" style="float:right; height:507px; width:1000px" /></p>

<p>This tutorial will introduce you to FCTM fragment centric topographical mapping of a single snapshot. For older version, click <a href="https://www.nyu.edu/projects/yzhang/AlphaSpace/" target="_blank">here</a>.</p>

<p>If you are familiar with AlphaSpace 1.0, here you will see some reference to its tutorial, otherwise you can ignore them.</p>


# 1. Program requirements

Python, SciPy, NumPy, Mdtraj, Jupyter

These dependencies should be installed automatically when you install AlphaSpace.

To view how to install, please proceed to [README.md](../README.md)

Now AlphaSpace 2.0 no longer requires Naccess for ASA calculation, it is replaced by [Shrake and Rupley](http://mdtraj.org/1.7.2/examples/solvent-accessible-surface-area.html).

# 2. Load PDB File

AlphaSpace 2.0 can process multiple snapshots from a simulation, but for now let's start with a single snapshot. We will use the mdm2-p53 complex, same as 1.0 tutorial.

AlphaSpace can map a complete protein surface or can be restricted to map just the interface between a receptor protein and a binding partner. The "receptor"/"protein" part and the "binder"/"ligand" part can be automatically identified, or manually set.

First Import the modules, you should see them

In [1]:
import alphaspace,mdtraj

Now we can load the [pdb file](mdm2_p53/mdm2_p53.pdb), and initiate our *Universe*. Remember you should set the path to your own protein in real use.

In [2]:
universe = alphaspace.AS_Universe(mdtraj.load('mdm2_p53/mdm2_p53.pdb'),guess_receptor_binder=True,guess_by_order=False)

<p><strong>AlphaSpace</strong>&nbsp;will automatically strip all non-ATOM/HETATM lines from the input PDB file, and additionally will strip all <strong>protons</strong>, water molecules, and Na+/Cl- ions. Then the receptor and binder are guessed by their number of atoms, the larger one as receptor (<code>guess_receptor_binder</code>). Alternatively, you can turn on <code>guess_by_order</code> (set the first one as receptor and second one as binder). You can see here from the atom number and residue number, the guess was correct.</p>

<div style="background:#eee; border:1px solid #ccc; padding:5px 10px"><code>universe = alphaspace.AS_Universe(mdtraj.load(&#39;mdm2_p53/mdm2_p53.pdb&#39;),guess_receptor_binder=True,guess_by_order=True)</code></div>

<p><strong>Different from AlphaSpace 1.0</strong>, you do not need to place TER between molecules.</p>


<h1>3. Understanding the Parameter file</h1>

<p>&nbsp;</p>

<p>AlphaSpace 1.0 uses a parameter file&nbsp;&quot;<em><u>AS_param.txt</u></em>&quot; and &quot;<u><em>AS_option.txt</em></u>&quot;, now they are combined into one single file <a href="../alphaspace/config.ini" target="_blank">config.ini</a>. You can checkout their details at the end of the document.&nbsp;They have the same content and you can make your own parameter file by adding parameter and option flag:&nbsp;</p>

<blockquote>
<p><code>[options] </code></p>

<p><code>output_dir = . </code></p>

<p><code>[parameters] </code></p>

<p><code>min_r = 3.2</code></p>
</blockquote>

<p>&nbsp;</p>

<p>Of particular interest are the options: &quot;<u><code>screen_by_face</code></u>&quot; and &quot;screen_by_lig_cntct&quot;. If only pockets in direct contact with the ligand are desired, set <u><code>screen_by_lig_cntct = True.</code></u> In order to expand the pocket selection to also include unoccupied pockets in the vicinity of the ligand, set <u><code>screen_by_lig_cntct = False </code></u>and set <u><code>screen_by_face = True.</code></u> This setting will select pockets, not by ligand contact, but if a percentage of atoms from the pocket atom list are within the &quot;interface&quot; atom list.<s> This option requires Naccess to be activated in order to calculate the interface atom list.</s></p>

<p>You can specify your own <em>config.ini&nbsp;</em>by changing the default config.ini file, it is located <a href="../aphaspace/config.ini" target="_blank">here</a>, or change it on the fly here in the notebook like this.</p>


In [3]:
universe.config.screen_by_lig_cntct = True

<h1>4. Runing AlphaSpace</h1>

<h2>4.1 <strong>Jupyter Notebook</strong></h2>

<p>You can alphaspace simply by calling:</p>

<p><code>universe.run_alphaspace()</code></p>

<p>This would process the snapshot(s) based on the configuration in the <code>config.ini</code>. If you change the config.ini after this point, you will need to re-run alphaspace for it to be active.</p>


In [4]:
universe.run_alphaspace()

0 snapshot processed


<h1>5.&nbsp;Pocket Analysis</h1>


<p><img alt="" src="https://www.nyu.edu/projects/yzhang/AlphaSpace/images/table_mdm2_2.png" style="float:right; height:554px; width:1000px" /></p>

<p>Above shows the pockets detected in alphaspace 1.0, and visualized in Chimera. In 2.0,&nbsp;you can <strong>iterate</strong> through pockets in a given snapshot by calling the <strong>.pockets</strong> method. For instance if you want to see pockets in snapshot 0:</p>

In [5]:
for pocket in universe.pockets(snapshot_idx=0):
    print(pocket)

<Pocket 1 in snapshot 0 with space 250 and 73% occupied>
<Pocket 2 in snapshot 0 with space 242 and 94% occupied>
<Pocket 3 in snapshot 0 with space 187 and 34% occupied>
<Pocket 4 in snapshot 0 with space 71 and 51% occupied>
<Pocket 5 in snapshot 0 with space 55 and 88% occupied>
<Pocket 6 in snapshot 0 with space 44 and 72% occupied>
<Pocket 7 in snapshot 0 with space 14 and 100% occupied>


You can also access the pocket properties such as:
occupancy,space,is_contact

In [6]:
for pocket in universe.pockets(snapshot_idx=0):
    print(pocket.occupancy,
          pocket.space,
          pocket.is_contact)

0.7218682455379769 249.9373348870114 True
0.9392116580132764 241.02131003800474 True
0.33135460177034765 186.89316884778023 True
0.5073220222498638 70.32360315295703 True
0.8768471705582687 54.50895613603793 True
0.7146894781987956 43.8709929452111 True
1.0 13.081765089579006 True


<h1>Options<a id="options" name="options"></a></h1>

<blockquote>
<pre>
get_face = True

###calculate the interface atom list

screen_by_face = False

###select pockets that include atoms from the interface atom list

screen_face_perc = 0.5

###minimum percentage of atoms that must be in the interface atom list

screen_by_lig_cntct = False

###select only pockets in &quot;contact&quot; with the ligand

expand_around_cntct = False

###expand selection to include all pockets overlapping the &quot;contact&quot; pockets

screen_by_score = False
min_score = 100

###select only pockets above a minimum score

screen_by_perc_rank = False
min_perc_rank = 0.9

###select only pockets above of minimum percentage ranking

screen_out_subsurf = False max_desolv_perc = 0.95
###screen out pocket if too much of the alpha-cluster ASA is desolvated by the pocket
###(i.e. pocket is beneath the surface) (more expensive calculation)

screen_by_res = False
res_file = screen_resID.pdb

###select all pockets containing any atom from any residue within the residue pdb file

pocket_communities = True

###calculate and output the pocket communities</pre>
</blockquote>


<h1>Parameters<a id="parameters" name="parameters"></a></h1>

<blockquote>
<pre>
min_r = 3.2 
max_r = 5.4

#minimum and maximum alpha-sphere radius for filtering alpha-spheres

clust_dist = 4.7

#distance parameter for average linkage clustering of alpha-atoms

#into alpha-clusters

min_num_alph = 1

#filter alpha-clusters by a minimum # of alpha-atoms

hit_dist = 1.6

#ligand/alpha-atom &quot;contact&quot; distance

core_cutoff = 100 
aux_cutoff = 30

#minimum pocket score for &quot;high-scoring&quot; and &quot;mid-scoring&quot; classification</pre>
</blockquote>
